In [1]:
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_jinhuijiu.csv')
data_rfe=np.array(data_rfe)
fea=data_rfe[:,:49]
target=data_rfe[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

# SVM-RFE_RELIEF

In [2]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,alpha,folds):
        #kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            if len(candi_fea)==optim_featur_num:
                return candi_fea
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(i)
            i-=1
            

In [3]:
rfe_relief_model=rfe_relief_SVM(x_train,y_train,0.0039,0.1)

In [4]:
aver_mean_lis1,F_measure_lis1=rfe_relief_model.featu_score(0.15,5)
print(aver_mean_lis1,F_measure_lis1)

[0.6104624458479044, 0.5191359152356595, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106] [0.7204719182913755

In [5]:
aver_mean_lis2,F_measure_lis2=rfe_relief_model.featu_score(0.25,5)
print(aver_mean_lis2,F_measure_lis2)

[0.6104624458479044, 0.5191359152356595, 0.507911425439741, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106] [0.7204719182913755,

In [6]:
aver_mean_lis3,F_measure_lis3=rfe_relief_model.featu_score(0.35,5)
print(aver_mean_lis3,F_measure_lis3)

[0.6104624458479044, 0.5191359152356595, 0.507911425439741, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106] [0.7204719182913755,

In [7]:
aver_mean_lis4,F_measure_lis4=rfe_relief_model.featu_score(0.45,5)
print(aver_mean_lis4,F_measure_lis4)

[0.6104624458479044, 0.5247481601336187, 0.5104624458479045, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106] [0.7204719182913755

In [8]:
aver_mean_lis5,F_measure_lis5=rfe_relief_model.featu_score(0.55,5)
print(aver_mean_lis5,F_measure_lis5)

[0.6104624458479044, 0.5247481601336187, 0.507911425439741, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106] [0.7204719182913755,

In [9]:
aver_mean_lis6,F_measure_lis6=rfe_relief_model.featu_score(0.65,5)
print(aver_mean_lis6,F_measure_lis6)

[0.6104624458479044, 0.5247481601336187, 0.507911425439741, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106] [0.7204719182913755,

In [10]:
aver_mean_lis7,F_measure_lis7=rfe_relief_model.featu_score(0.75,5)
print(aver_mean_lis7,F_measure_lis7)

[0.6104624458479044, 0.5191359152356595, 0.5068910172764758, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106] [0.7204719182913755

In [11]:
aver_mean_lis8,F_measure_lis8=rfe_relief_model.featu_score(0.85,5)
print(aver_mean_lis8,F_measure_lis8)

[0.6104624458479044, 0.6436387598517668, 0.577297875672008, 0.5905749256224229, 0.5109726499295371, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106] [0.7204719182913755,

In [12]:
aver_mean_lis9,F_measure_lis9=rfe_relief_model.featu_score(0.95,5)
print(aver_mean_lis9,F_measure_lis9)

[0.6104624458479044, 0.6436387598517668, 0.5165848948274963, 0.5318910172764758, 0.5063808131948432, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106, 0.5058706091132106] [0.7204719182913755

In [24]:
np.max(aver_mean_lis1[:8]),np.max(aver_mean_lis2[:8]),np.max(aver_mean_lis3[:8]),np.max(aver_mean_lis4[:8]),np.max(aver_mean_lis5[:8]),np.max(aver_mean_lis6[:8]),np.max(aver_mean_lis7[:8]),np.max(aver_mean_lis8[:8]),np.max(aver_mean_lis9[:8])

(0.6104624458479044,
 0.6104624458479044,
 0.6104624458479044,
 0.6104624458479044,
 0.6104624458479044,
 0.6104624458479044,
 0.6104624458479044,
 0.6436387598517668,
 0.6436387598517668)

In [25]:
np.argmax(aver_mean_lis9[:8])

1

In [26]:
aver_mean_lis9[:8]

[0.6104624458479044,
 0.6436387598517668,
 0.5165848948274963,
 0.5318910172764758,
 0.5063808131948432,
 0.5058706091132106,
 0.5058706091132106,
 0.5058706091132106]

# SVM-RFE

In [14]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,folds):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            if len(candi_fea)==featur_num:
                return candi_fea
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                #print(scor_fea)
                rfe_score[k]=scor_fea
                k=k+1     
            #print(rfe_score)
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(1)

In [15]:
rfe_SVM_model=rfe_SVM(x_train,y_train,0.0039,0.1)
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis=rfe_SVM_model.featu_score(5)

In [16]:
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis

([0.6104624458479044,
  0.5191359152356595,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.5058706091132106,
  0.505870

# SVM-RELIEF

In [17]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class relief_SVM:
    def __init__(self,x_train,y_train,C,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
        self.Gamma=gamma
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        featu_num=self.X_train.shape[1]
        relief_fea_score=reliefF.reliefF(self.X_train,self.Y_train)
        relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        for i in range(featu_num):
            estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
            candi_fea=relief_candi_fea[:(i+1)]
            aver=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i]=aver
            F_lis[i]=F1 
        return aver_lis,F_lis

In [18]:
relief_SVM_model=relief_SVM(x_train,y_train,0.0039,0.1)
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis=relief_SVM_model.featu_score(5)

In [19]:
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis

([0.5706978443551334,
  0.6008546897019678,
  0.5824912573725143,
  0.6176914243958452,
  0.5906414739808967,
  0.541648833446422,
  0.524283626494076,
  0.5058940967691424,
  0.48802390521425953,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704,
  0.48751370113262704

In [27]:
jinhui_accuracy=[]
jinhui_accuracy.append(aver_mean_lis1)
jinhui_accuracy.append(aver_mean_lis2)
jinhui_accuracy.append(aver_mean_lis3)
jinhui_accuracy.append(aver_mean_lis4)
jinhui_accuracy.append(aver_mean_lis5)
jinhui_accuracy.append(aver_mean_lis6)
jinhui_accuracy.append(aver_mean_lis7)
jinhui_accuracy.append(aver_mean_lis8)
jinhui_accuracy.append(aver_mean_lis9)
jinhui_accuracy.append(rfe_SVM_aver_mean_lis)
jinhui_accuracy.append(relief_SVM_aver_mean_lis)

In [28]:
colu=[i for i in range(1,50)]
jinhui_acurracy_dataframe=pd.DataFrame(columns=colu,data=np.array(jinhui_accuracy))
jinhui_acurracy_dataframe.to_csv("jinhui_accuracy.csv")

In [29]:
jinhui_F_measure=[]
jinhui_F_measure.append(F_measure_lis1)
jinhui_F_measure.append(F_measure_lis2)
jinhui_F_measure.append(F_measure_lis3)
jinhui_F_measure.append(F_measure_lis4)
jinhui_F_measure.append(F_measure_lis5)
jinhui_F_measure.append(F_measure_lis6)
jinhui_F_measure.append(F_measure_lis7)
jinhui_F_measure.append(F_measure_lis8)
jinhui_F_measure.append(F_measure_lis9)
jinhui_F_measure.append(rfe_SVM_F_measure_lis)
jinhui_F_measure.append(relief_SVM_F_measure_lis)

In [30]:
colu=[i for i in range(1,50)]
jinhui_F_measure_dataframe=pd.DataFrame(columns=colu,data=np.array(jinhui_F_measure))
jinhui_F_measure_dataframe.to_csv("jinhui_F_measure.csv")